In [12]:
import os
os.chdir("../")
%pwd

'/home/anonymous/PycharmProjects/MLOps_DS'

In [13]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: Path

In [14]:
from mlds.constants import *
from mlds.utils.common import read_yaml, create_directories

In [15]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema=schema,
        )

        return data_validation_config

In [16]:
import os
from mlds import logger
# from mlds.entity.config_entity import DataValidationConfig
import pandas as pd


class DataValiadtion:
    def __init__(self, config: DataValidationConfig):
        self.config = config


    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e


In [17]:
try:
    config = ConfigurationManager()
    data_val_config = config.get_data_validation_config()
    data_val = DataValiadtion(config=data_val_config)
    data_val.validate_all_columns()
    
except Exception as e:
    raise e

[2023-09-08 13:48:18,179: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-09-08 13:48:18,182: INFO: common: yaml file: params.yaml loaded successfully]
[2023-09-08 13:48:18,186: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-09-08 13:48:18,188: INFO: common: created directory at: artifacts]
[2023-09-08 13:48:18,190: INFO: common: created directory at: artifacts/data_validation]
